In [18]:
import sys
sys.path.append('F:\Cambridge\Project\MHMC-for-VAE\change_of_variable')
sys.path.append('F:\Cambridge\Project\MHMC-for-VAE\hmc_pytorch')
from change_of_variable_pytorch import * 
from hmc_base_pytorch import *
from hmc_unconstrained_pytorch import *
import numpy as np
import math
import torch
import torch.nn as nn

In [19]:
def log_prior(z):
    dim = z.shape[0]
    mean = torch.zeros(dim).cuda()
    cov = 2*torch.eye(dim).cuda()
    m = MultivariateNormal(mean, cov)
    m.requires_grad=True
    return m.log_prob(z)

In [20]:
def multivariate_normal_diagonal_logpdf(mean, cov_diag, x):
     # number of samples
    k = x.shape[0] # dimension
    #print(x.is_cuda)
    #print(mean.is_cuda)
    #print(cov_diag.is_cuda)
    t1 = -0.5*(x - mean)*(1/cov_diag)*(x-mean)
    t1 = torch.sum(t1, dim=1)
    t2 = 0.5*k*torch.log(torch.tensor([2*math.pi]).cuda()) + 0.5*torch.sum(torch.log(cov_diag),dim=1)
    return t1 - t2

In [21]:
z_cuda = torch.zeros(2, requires_grad=True).cuda()
mean = torch.zeros([1,2], requires_grad=True).cuda()
cov_diag = 2*torch.ones([1,2], requires_grad=True).cuda()
-multivariate_normal_diagonal_logpdf(mean, cov_diag, z_cuda)

tensor([ 2.5310], device='cuda:0')

In [5]:
log_prior(z_cuda)

tensor(-2.5310, device='cuda:0')

In [22]:
def energy(z, cache):
    z.retain_grad()
    z_cuda = z.cuda()
    mean = torch.ones([1,2], requires_grad=True).cuda()
    cov_diag = torch.ones([1,2], requires_grad=True).cuda()
    res = -multivariate_normal_diagonal_logpdf(mean, cov_diag, z_cuda)
    #print("energy: "+str(res))
    return res

In [23]:
sampler = IsotropicHmcSampler(energy, energy_grad=None, prng=None,
                                          mom_resample_coeff=1., dtype=np.float64)

In [24]:
mass_matrix = torch.eye(2).cuda()
mom = torch.tensor([1.,1.], requires_grad=True).cuda()
cache = {}
#pos_samples, mom_samples, ratio = sampler.get_samples(init, 0.001, 5, 2, mass = mass_matrix, mom = mom)
#get_samples(self, pos, dt, n_step_per_sample, n_sample, mass, mom=None):
#pos_samples

In [25]:
init = torch.zeros(2, requires_grad=True)
tmp = torch.tensor(init.data, requires_grad=True)
init_cuda = init.cuda()
tmp_cuda = tmp.cuda()
pos_samples, mom_samples, ratio = sampler.get_samples(init_cuda, 0.002, 5, 2, mass = mass_matrix, mom = mom)
zt = pos_samples[1]
#pos_samples
zt

pos: tensor([ 0.,  0.])
grad: tensor([-1., -1.])
loop pos: tensor(1.00000e-03 *
       [ 2.0020,  2.0020], device='cuda:0')
loop grad: tensor([-0.9980, -0.9980], device='cuda:0')
loop pos: tensor(1.00000e-03 *
       [ 4.0080,  4.0080], device='cuda:0')
loop grad: tensor([-0.9960, -0.9960], device='cuda:0')
loop pos: tensor(1.00000e-03 *
       [ 6.0180,  6.0180], device='cuda:0')
loop grad: tensor([-0.9940, -0.9940], device='cuda:0')
loop pos: tensor(1.00000e-03 *
       [ 8.0319,  8.0319], device='cuda:0')
loop grad: tensor([-0.9920, -0.9920], device='cuda:0')


tensor(1.00000e-02 *
       [ 1.0050,  1.0050])

In [14]:
#init = torch.zeros(2, requires_grad=True)
#init_cuda = init.cuda()
#L = torch.mean(pos_samples)#+torch.mean(init)
L = torch.mean(zt)
L.backward()
print(init.grad)
#init.grad.data.zero_()

tensor([ 0.5000,  0.5000])


In [25]:
L

tensor(1.0040)

In [66]:
torch.mean(pos_samples)

tensor(1.)

In [104]:
x = torch.tensor(2, requires_grad=True)
y = x**2
z = 2*x
L = y+z
L.backward()
x.grad

tensor(6)

In [14]:
init = torch.ones(2, requires_grad=True).cuda()
mass_matrix = torch.eye(2).cuda()
mom = torch.tensor([0.1,0.1], requires_grad=True).cuda()
cache = {}
pos_samples, mom_samples, ratio = sampler.get_samples(init, 0.001, 5, 3, mass = mass_matrix, mom = mom)
#get_samples(self, pos, dt, n_step_per_sample, n_sample, mass, mom=None):
pos_samples

========= Sample: 1
pos: tensor([ 1.,  1.])
grad: tensor([ 1.,  1.])
loop pos: tensor([ 1.0001,  1.0001], device='cuda:0')
loop grad: tensor([ 1.0001,  1.0001], device='cuda:0')
loop pos: tensor([ 1.0002,  1.0002], device='cuda:0')
loop grad: tensor([ 1.0002,  1.0002], device='cuda:0')
loop pos: tensor([ 1.0003,  1.0003], device='cuda:0')
loop grad: tensor([ 1.0003,  1.0003], device='cuda:0')
loop pos: tensor([ 1.0004,  1.0004], device='cuda:0')
loop grad: tensor([ 1.0004,  1.0004], device='cuda:0')
hamiltonian_p: tensor([ 2.8479], device='cuda:0')
========= Sample: 2
pos: tensor([ 1.0005,  1.0005])
grad: tensor([ 1.0005,  1.0005])
loop pos: tensor([ 0.9996,  1.0003], device='cuda:0')
loop grad: tensor([ 0.9996,  1.0003], device='cuda:0')
loop pos: tensor([ 0.9987,  1.0001], device='cuda:0')
loop grad: tensor([ 0.9987,  1.0001], device='cuda:0')
loop pos: tensor([ 0.9978,  1.0000], device='cuda:0')
loop grad: tensor([ 0.9978,  1.0000], device='cuda:0')
loop pos: tensor([ 0.9969,  0.999

tensor([[ 1.0000,  1.0000],
        [ 1.0005,  1.0005],
        [ 0.9960,  0.9996]])

In [18]:
init = torch.zeros(2, requires_grad=True).cuda()
mass_matrix = torch.eye(2).cuda()
mom = torch.tensor([0.1,0.1], requires_grad=True).cuda()
cache = {}
pos_samples, mom_samples, ratio = sampler.get_samples(init, 0.05, 5, 2, mass = mass_matrix, mom = mom)
#get_samples(self, pos, dt, n_step_per_sample, n_sample, mass, mom=None):
pos_samples

pos: tensor([ 0.,  0.])
grad: tensor([ 0.,  0.])
loop grad: tensor([ 0.,  0.], device='cuda:0')
loop grad: tensor([ 0.,  0.], device='cuda:0')
loop grad: tensor([ 0.,  0.], device='cuda:0')
loop grad: tensor([ 0.,  0.], device='cuda:0')
mom: tensor([-0.4941, -1.3525])


tensor(1.00000e-02 *
       [[ 0.0000,  0.0000],
        [ 2.5000,  2.5000]])

In [8]:
ratio

0.6666666666666667

In [39]:
 def gradient(pos, cache):
    #pos.retain_grad()
    x = energy(pos, cache)
    print("x: "+str(x))
    #x.backward(retain_graph=True)
    x.backward()
    g = pos.grad.clone()
    #pos.grad.zero_()
    #print("g:"+str(g))
    #pos.grad.data.zero_()
    return g

In [119]:
 def gradient(pos, cache):
    #print("------calling energy function------")
    pos.retain_grad()
    x = energy(pos, cache)
    print("x: "+str(x))
    x.backward(retain_graph=True)

    g = pos.grad.clone()
    #pos.grad.zero_()
    #print("g:"+str(g))
    pos.grad.data.zero_()
    return g

In [40]:
pos = torch.zeros(2, requires_grad=True).cuda()
gradient(pos, cache)

x: tensor([ 1.8379], device='cuda:0')


tensor([ 0.,  0.], device='cuda:0')

In [5]:
np.var(z1_list)
#plt.hist(z1_list)

0.043755842097883399

In [4]:
def energy(z, cache):
    #z.retain_grad()
    z_cuda = z.cuda()
    mean = 3*torch.ones([1,1], requires_grad=True).cuda()
    cov_diag = torch.ones([1,1], requires_grad=True).cuda()
    res = -multivariate_normal_diagonal_logpdf(mean, cov_diag, z_cuda)
    #print("energy: "+str(res))
    return res

In [ ]:
#init = torch.zeros(1, requires_grad=True).cuda()
#mass_matrix = torch.eye(2).cuda()
#mass_matrix = torch.tensor([1.]).cuda()
#mom = torch.tensor([0.1]).cuda()
sampler = IsotropicHmcSampler(energy, energy_grad=None, prng=None,
                                          mom_resample_coeff=1., dtype=np.float64)

z1_list = []
#z2_list = []
m = Normal(torch.tensor([0.0]), torch.tensor([1.0]))

init = torch.ones(1, requires_grad=True).cuda()
mass_matrix = torch.tensor([1.]).cuda()
pos_samples, mom_samples, ratio = sampler.get_samples(init, 0.001, 5, 1000, mass_matrix)
for i in range(1000):
    z1_list.append(pos_samples[i].item())
    
"""
for i in range(500):
    init = torch.ones(1, requires_grad=True).cuda()
    mass_matrix = torch.tensor([1.]).cuda()
    mom = m.sample().cuda()
    pos_samples, mom_samples, ratio = sampler.get_samples(init, 0.01, 10, 2, mass_matrix,mom)
    z1_list.append(pos_samples[1][0].item())
"""
"""
for i in range(100):
    z1_list.append(pos_samples[i].item())
#z1_list.append(pos_samples[1][0].item())
    #z2_list.append(pos_samples[1][1].item())
"""

Sample: 1
hamiltonian_p: tensor([ 2.9737], device='cuda:0')
Sample: 2
hamiltonian_p: tensor([ 3.2879], device='cuda:0')
Sample: 3
hamiltonian_p: tensor([ 2.9138], device='cuda:0')
Sample: 4
hamiltonian_p: tensor([ 2.9368], device='cuda:0')
Sample: 5
hamiltonian_p: tensor([ 3.0230], device='cuda:0')
Sample: 6
hamiltonian_p: tensor([ 5.4465], device='cuda:0')
Sample: 7
hamiltonian_p: tensor([ 2.9492], device='cuda:0')
Sample: 8
hamiltonian_p: tensor([ 3.4978], device='cuda:0')
Sample: 9
hamiltonian_p: tensor([ 3.9959], device='cuda:0')
Sample: 10
hamiltonian_p: tensor([ 2.9196], device='cuda:0')
Sample: 11
hamiltonian_p: tensor([ 4.3344], device='cuda:0')
Sample: 12
hamiltonian_p: tensor([ 3.6956], device='cuda:0')
Sample: 13
hamiltonian_p: tensor([ 2.9682], device='cuda:0')
Sample: 14
hamiltonian_p: tensor([ 2.9347], device='cuda:0')
Sample: 15
hamiltonian_p: tensor([ 2.9932], device='cuda:0')
Sample: 16
hamiltonian_p: tensor([ 4.5316], device='cuda:0')
Sample: 17
hamiltonian_p: tensor(

hamiltonian_p: tensor([ 2.9126], device='cuda:0')
Sample: 136
hamiltonian_p: tensor([ 3.8037], device='cuda:0')
Sample: 137
hamiltonian_p: tensor([ 3.1953], device='cuda:0')
Sample: 138
hamiltonian_p: tensor([ 2.9078], device='cuda:0')
Sample: 139
hamiltonian_p: tensor([ 3.5497], device='cuda:0')
Sample: 140
hamiltonian_p: tensor([ 2.9768], device='cuda:0')
Sample: 141
hamiltonian_p: tensor([ 3.3694], device='cuda:0')
Sample: 142
hamiltonian_p: tensor([ 3.3325], device='cuda:0')
Sample: 143
hamiltonian_p: tensor([ 3.7775], device='cuda:0')
Sample: 144
hamiltonian_p: tensor([ 3.7190], device='cuda:0')
Sample: 145
hamiltonian_p: tensor([ 4.4636], device='cuda:0')
Sample: 146
hamiltonian_p: tensor([ 2.9357], device='cuda:0')
Sample: 147
hamiltonian_p: tensor([ 3.3721], device='cuda:0')
Sample: 148
hamiltonian_p: tensor([ 3.5319], device='cuda:0')
Sample: 149
hamiltonian_p: tensor([ 3.1502], device='cuda:0')
Sample: 150
hamiltonian_p: tensor([ 2.9169], device='cuda:0')
Sample: 151
hamilton

hamiltonian_p: tensor([ 3.5203], device='cuda:0')
Sample: 269
hamiltonian_p: tensor([ 3.5286], device='cuda:0')
Sample: 270
hamiltonian_p: tensor([ 3.1765], device='cuda:0')
Sample: 271
hamiltonian_p: tensor([ 3.8524], device='cuda:0')
Sample: 272
hamiltonian_p: tensor([ 5.1615], device='cuda:0')
Sample: 273
hamiltonian_p: tensor([ 10.1053], device='cuda:0')
Sample: 274
hamiltonian_p: tensor([ 3.4791], device='cuda:0')
Sample: 275
hamiltonian_p: tensor([ 3.2268], device='cuda:0')
Sample: 276
hamiltonian_p: tensor([ 3.1243], device='cuda:0')
Sample: 277
hamiltonian_p: tensor([ 3.1282], device='cuda:0')
Sample: 278
hamiltonian_p: tensor([ 3.1347], device='cuda:0')
Sample: 279
hamiltonian_p: tensor([ 5.6972], device='cuda:0')
Sample: 280
hamiltonian_p: tensor([ 3.2087], device='cuda:0')
Sample: 281
hamiltonian_p: tensor([ 4.3681], device='cuda:0')
Sample: 282
hamiltonian_p: tensor([ 3.2290], device='cuda:0')
Sample: 283
hamiltonian_p: tensor([ 4.3464], device='cuda:0')
Sample: 284
hamilto

hamiltonian_p: tensor([ 3.4127], device='cuda:0')
Sample: 402
hamiltonian_p: tensor([ 4.2091], device='cuda:0')
Sample: 403
hamiltonian_p: tensor([ 3.5347], device='cuda:0')
Sample: 404
hamiltonian_p: tensor([ 3.2421], device='cuda:0')
Sample: 405
hamiltonian_p: tensor([ 3.2996], device='cuda:0')
Sample: 406
hamiltonian_p: tensor([ 7.4428], device='cuda:0')
Sample: 407
hamiltonian_p: tensor([ 4.3538], device='cuda:0')
Sample: 408
hamiltonian_p: tensor([ 3.2759], device='cuda:0')
Sample: 409
hamiltonian_p: tensor([ 3.2882], device='cuda:0')
Sample: 410
hamiltonian_p: tensor([ 3.2914], device='cuda:0')
Sample: 411
hamiltonian_p: tensor([ 3.5841], device='cuda:0')
Sample: 412
hamiltonian_p: tensor([ 4.4574], device='cuda:0')
Sample: 413
hamiltonian_p: tensor([ 3.5615], device='cuda:0')
Sample: 414
hamiltonian_p: tensor([ 4.6428], device='cuda:0')
Sample: 415
hamiltonian_p: tensor([ 3.7172], device='cuda:0')
Sample: 416
hamiltonian_p: tensor([ 4.1033], device='cuda:0')
Sample: 417
hamilton

hamiltonian_p: tensor([ 3.0643], device='cuda:0')
Sample: 535
hamiltonian_p: tensor([ 3.3536], device='cuda:0')
Sample: 536
hamiltonian_p: tensor([ 3.0467], device='cuda:0')
Sample: 537
hamiltonian_p: tensor([ 6.3289], device='cuda:0')
Sample: 538
hamiltonian_p: tensor([ 3.6334], device='cuda:0')
Sample: 539
hamiltonian_p: tensor([ 3.8191], device='cuda:0')
Sample: 540
hamiltonian_p: tensor([ 3.7537], device='cuda:0')
Sample: 541
hamiltonian_p: tensor([ 4.2122], device='cuda:0')
Sample: 542
hamiltonian_p: tensor([ 3.1473], device='cuda:0')
Sample: 543
hamiltonian_p: tensor([ 2.9888], device='cuda:0')
Sample: 544
hamiltonian_p: tensor([ 3.1113], device='cuda:0')
Sample: 545
hamiltonian_p: tensor([ 3.9738], device='cuda:0')
Sample: 546
hamiltonian_p: tensor([ 2.9688], device='cuda:0')
Sample: 547
hamiltonian_p: tensor([ 3.6565], device='cuda:0')
Sample: 548
hamiltonian_p: tensor([ 2.9568], device='cuda:0')
Sample: 549
hamiltonian_p: tensor([ 2.9982], device='cuda:0')
Sample: 550
hamilton

hamiltonian_p: tensor([ 3.5216], device='cuda:0')
Sample: 668
hamiltonian_p: tensor([ 5.1208], device='cuda:0')
Sample: 669
hamiltonian_p: tensor([ 4.1398], device='cuda:0')
Sample: 670
hamiltonian_p: tensor([ 2.9877], device='cuda:0')
Sample: 671
hamiltonian_p: tensor([ 3.7226], device='cuda:0')
Sample: 672
hamiltonian_p: tensor([ 3.2359], device='cuda:0')
Sample: 673
hamiltonian_p: tensor([ 3.7468], device='cuda:0')
Sample: 674
hamiltonian_p: tensor([ 4.8730], device='cuda:0')
Sample: 675
hamiltonian_p: tensor([ 3.0103], device='cuda:0')
Sample: 676
hamiltonian_p: tensor([ 2.9430], device='cuda:0')
Sample: 677
hamiltonian_p: tensor([ 2.9401], device='cuda:0')
Sample: 678
hamiltonian_p: tensor([ 4.1542], device='cuda:0')
Sample: 679
hamiltonian_p: tensor([ 4.0938], device='cuda:0')
Sample: 680
hamiltonian_p: tensor([ 3.1578], device='cuda:0')
Sample: 681
hamiltonian_p: tensor([ 3.6968], device='cuda:0')
Sample: 682
hamiltonian_p: tensor([ 2.9156], device='cuda:0')
Sample: 683
hamilton

hamiltonian_p: tensor([ 3.2595], device='cuda:0')
Sample: 801
hamiltonian_p: tensor([ 2.9414], device='cuda:0')
Sample: 802
hamiltonian_p: tensor([ 4.2242], device='cuda:0')
Sample: 803
hamiltonian_p: tensor([ 2.9239], device='cuda:0')
Sample: 804
hamiltonian_p: tensor([ 3.0083], device='cuda:0')
Sample: 805
hamiltonian_p: tensor([ 3.2869], device='cuda:0')
Sample: 806
hamiltonian_p: tensor([ 3.1155], device='cuda:0')
Sample: 807
hamiltonian_p: tensor([ 2.9370], device='cuda:0')
Sample: 808
hamiltonian_p: tensor([ 3.6700], device='cuda:0')
Sample: 809
hamiltonian_p: tensor([ 2.9353], device='cuda:0')
Sample: 810
hamiltonian_p: tensor([ 2.9378], device='cuda:0')
Sample: 811
hamiltonian_p: tensor([ 3.3564], device='cuda:0')
Sample: 812
hamiltonian_p: tensor([ 2.9669], device='cuda:0')
Sample: 813
hamiltonian_p: tensor([ 2.9223], device='cuda:0')
Sample: 814
hamiltonian_p: tensor([ 3.7043], device='cuda:0')
Sample: 815
hamiltonian_p: tensor([ 3.8373], device='cuda:0')
Sample: 816
hamilton

hamiltonian_p: tensor([ 2.8946], device='cuda:0')
Sample: 934
hamiltonian_p: tensor([ 2.9975], device='cuda:0')
Sample: 935
hamiltonian_p: tensor([ 3.7964], device='cuda:0')
Sample: 936
hamiltonian_p: tensor([ 4.0515], device='cuda:0')
Sample: 937
hamiltonian_p: tensor([ 3.1016], device='cuda:0')
Sample: 938
hamiltonian_p: tensor([ 3.0754], device='cuda:0')
Sample: 939
hamiltonian_p: tensor([ 3.0732], device='cuda:0')
Sample: 940
hamiltonian_p: tensor([ 3.0766], device='cuda:0')
Sample: 941
hamiltonian_p: tensor([ 2.9012], device='cuda:0')
Sample: 942
hamiltonian_p: tensor([ 2.8997], device='cuda:0')
Sample: 943
hamiltonian_p: tensor([ 2.9754], device='cuda:0')
Sample: 944
hamiltonian_p: tensor([ 3.0043], device='cuda:0')
Sample: 945
hamiltonian_p: tensor([ 5.2676], device='cuda:0')
Sample: 946
hamiltonian_p: tensor([ 4.7960], device='cuda:0')
Sample: 947
hamiltonian_p: tensor([ 2.9259], device='cuda:0')
Sample: 948
hamiltonian_p: tensor([ 3.1165], device='cuda:0')
Sample: 949
hamilton

In [1]:
np.var(z1_list)
#plt.hist(z1_list)
#np.mean(z1_list)
#ratio

NameError: name 'np' is not defined

In [ ]:
#h = plt.hist2d(z1_list, z2_list, bins = [100, 100])

In [4]:
mean = torch.tensor([[0.,0.,0.],[0.,0.,0.]]).cuda()
cov_diag = torch.tensor([[1.,1.,1.],[1.,1.,1.]]).cuda()
x = torch.tensor([[5.,0.5,0.],[1.,1.,0.]]).cuda()
multivariate_normal_diagonal_logpdf(mean, cov_diag, x)

tensor([-15.3818,  -3.7568], device='cuda:0')

In [20]:
mean.shape
torch.diag(cov_diag[0])

tensor([[ 1.,  0.,  0.],
        [ 0.,  1.,  0.],
        [ 0.,  0.,  1.]], device='cuda:0')

In [ ]:
def energy(y,cache):
    mean = 
    multivariate_normal_diagonal_logpdf(mean, cov_diag, x)

In [15]:
mean = torch.tensor([[1.,0.,0.],[0.,0.,0.]], requires_grad=True)
mean.shape

torch.Size([2, 3])

In [18]:
res = torch.sum(mean, dim=1)
res.shape

torch.Size([2])

In [17]:
res.backward()

RuntimeError: grad can be implicitly created only for scalar outputs